In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/24 20:48:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## GroupBy

In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [4]:
df_green.registerTempTable("green")

/home/froukje/spark/spark-3.2.3-bin-hadoop3.2/python/pyspark/sql/dataframe.py:138: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn(


In [16]:
df_green_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    -- Revenue calculation 
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records 
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [17]:
df_green_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   7| 769.7299999999996|            45|
|2020-01-01 00:00:00|  17|195.03000000000003|             9|
|2020-01-01 00:00:00|  18|               7.8|             1|
|2020-01-01 00:00:00|  22|              15.8|             1|
|2020-01-01 00:00:00|  24|              87.6|             3|
|2020-01-01 00:00:00|  25| 531.0000000000002|            26|
|2020-01-01 00:00:00|  29|              61.3|             1|
|2020-01-01 00:00:00|  32| 68.94999999999999|             2|
|2020-01-01 00:00:00|  33|317.27000000000004|            11|
|2020-01-01 00:00:00|  35|            129.96|             5|
|2020-01-01 00:00:00|  36|295.34000000000003|            11|
|2020-01-01 00:00:00|  37|            175.67|             6|
|2020-01-01 00:00:00|  38| 98.78999999999999|             2|
|2020-01-01 00:00:00|  4

In [26]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode="overwrite")

In [19]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')
df_yellow.registerTempTable("yellow")

In [20]:
df_yellow_revenue = spark.sql("""
SELECT 
    -- Reveneue grouping 
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,

    -- Revenue calculation 
    SUM(total_amount) AS amount,
    COUNT(1) AS number_records 
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
ORDER BY
    1, 2
""")

In [21]:
df_yellow_revenue.show()

+-------------------+----+------------------+--------------+
|               hour|zone|            amount|number_records|
+-------------------+----+------------------+--------------+
|2020-01-01 00:00:00|   3|              25.0|             1|
|2020-01-01 00:00:00|   4|1004.2999999999998|            57|
|2020-01-01 00:00:00|   7|455.17000000000013|            38|
|2020-01-01 00:00:00|  10|             42.41|             2|
|2020-01-01 00:00:00|  12|             107.0|             6|
|2020-01-01 00:00:00|  13|1214.7999999999993|            56|
|2020-01-01 00:00:00|  14|               8.8|             1|
|2020-01-01 00:00:00|  15|             34.09|             1|
|2020-01-01 00:00:00|  17|220.21000000000004|             8|
|2020-01-01 00:00:00|  18|               5.8|             1|
|2020-01-01 00:00:00|  24| 754.9500000000003|            45|
|2020-01-01 00:00:00|  25|            324.35|            16|
|2020-01-01 00:00:00|  32|              18.0|             1|
|2020-01-01 00:00:00|  3

In [28]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode="overwrite")

## Joins

In [31]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_records', 'yellow_number_records')

In [32]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [33]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|   3|              null|                null|              25.0|                    1|
|2020-01-01 00:00:00|   4|              null|                null|1004.2999999999998|                   57|
|2020-01-01 00:00:00|   7| 769.7299999999996|                  45|455.17000000000013|                   38|
|2020-01-01 00:00:00|  12|              null|                null|             107.0|                    6|
|2020-01-01 00:00:00|  37|            175.67|                   6|            161.61|                    7|
|2020-01-01 00:00:00|  40|168.97999999999996|                   8|             89.97|                    5|
|2020-01-01 00:00:00|  45|  

In [34]:
df_join.write.parquet('data/report/revenue/total')

In [39]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [38]:
df_join = spark.read.parquet('data/report/revenue/total')

In [41]:
df_zones = spark.read.parquet('zones/')

In [42]:
df_zones

DataFrame[LocationID: string, Borough: string, Zone: string, service_zone: string]

In [45]:
df_result = df_join.join(df_zones, df_join.zone == df_zones.LocationID)

In [46]:
df_result

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint, LocationID: string, Borough: string, Zone: string, service_zone: string]

In [48]:
df_result.drop('LocationID', 'zone').write.parquet('tmp/revenue-zones')